In [2]:
import pandas as pd
df=pd.read_csv("dataset_base_features.csv")

In [4]:
df

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,stock_final,cat1,cat2,...,is_max_3,is_max_6,is_max_12,is_max_24,is_max_36,año,mes,cuatrimestre,mes_absoluto,cos_mes
0,10004,20001,2017-01-01,0.0,9.0,184.72927,184.72927,0.00000,1,10,...,1,1,1,1,1,2017,1,1,0,0.866025
1,10003,20001,2017-01-01,0.0,17.0,143.49426,143.49426,0.00000,1,10,...,0,0,0,0,0,2017,1,1,0,0.866025
2,10002,20001,2017-01-01,0.0,17.0,38.68301,35.72806,0.00000,1,10,...,0,0,0,0,0,2017,1,1,0,0.866025
3,10001,20001,2017-01-01,0.0,11.0,99.43861,99.43861,0.00000,1,10,...,0,0,0,0,0,2017,1,1,0,0.866025
4,10084,20001,2017-01-01,0.0,1.0,0.22386,0.22386,0.00000,1,10,...,0,0,0,0,0,2017,1,1,0,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9460975,10159,21276,2019-12-01,0.0,0.0,0.00000,0.00000,1.05592,2,6,...,0,0,0,0,0,2019,12,3,35,1.000000
9460976,10101,21276,2019-12-01,0.0,0.0,0.00000,0.00000,1.05592,2,6,...,1,0,0,0,0,2019,12,3,35,1.000000
9460977,10154,21276,2019-12-01,0.0,0.0,0.00000,0.00000,1.05592,2,6,...,1,0,0,0,0,2019,12,3,35,1.000000
9460978,10052,21276,2019-12-01,0.0,1.0,0.00594,0.00594,1.05592,2,6,...,1,1,1,0,0,2019,12,3,35,1.000000


In [30]:
#ESTO NO VA. ESTÁ TRAÍDO SOLO PARA TENERLO DISPONIBILIZADO Y ALCANZABLE EN CASO DE AVANZAR CON SEGUNDA ETAPA DE EXPERIMENTACIÓN.

# Paso 1: Agrupar por par product_id + customer_id y sumar tn
pares = df.groupby(['product_id', 'customer_id'], as_index=False)['tn'].sum()

# Paso 2: Ordenar de mayor a menor según tn
pares = pares.sort_values(by='tn', ascending=False)

# Paso 3: Calcular proporción acumulada
pares['acumulado'] = pares['tn'].cumsum()
total_tn = pares['tn'].sum()
pares['proporcion'] = pares['acumulado'] / total_tn

# Paso 4: Asignar bloque
def asignar_bloque(p):
    if p <= 0.40:
        return 'bloque_40'
    elif p <= 0.70:
        return 'bloque_30'
    elif p <= 0.90:
        return 'bloque_20'
    else:
        return 'bloque_10'

pares['bloque'] = pares['proporcion'].apply(asignar_bloque)

# Paso 5: Unir esta info al df original
df = df.merge(pares[['product_id', 'customer_id', 'bloque']], on=['product_id', 'customer_id'], how='left')

# Paso 6: Filtrar por bloque
df_40 = df[df['bloque'] == 'bloque_40']
df_30 = df[df['bloque'] == 'bloque_30']
df_20 = df[df['bloque'] == 'bloque_20']
df_10 = df[df['bloque'] == 'bloque_10']

# Paso 7: Control final
print(f"df_40: {df_40['tn'].sum():,.2f} tn")
print(f"df_30: {df_30['tn'].sum():,.2f} tn")
print(f"df_20: {df_20['tn'].sum():,.2f} tn")
print(f"df_10: {df_10['tn'].sum():,.2f} tn")
print(f"Total control: {df['tn'].sum():,.2f} tn")


df_40: 448,884.16 tn
df_30: 336,915.04 tn
df_20: 224,541.81 tn
df_10: 112,264.22 tn
Total control: 1,122,605.22 tn


In [ ]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

# Cargar datos
df = pd.read_csv("dataset_base_features.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# Pivotear: filas = productos, columnas = fechas
df_pivot = df.pivot_table(index='product_id', columns='periodo', values='tn', aggfunc='sum').fillna(0)

# Asegurar formato [n_series, n_timestamps, 1]
X = df_pivot.values[:, :, np.newaxis]

# Normalizar (opcional pero recomendado para DTW)
scaler = TimeSeriesScalerMeanVariance()
X_scaled = scaler.fit_transform(X)

# Clustering con DTW
n_clusters = 5  
km_dtw = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=42)
labels = km_dtw.fit_predict(X_scaled)

# Guardar resultado
df_clusters = pd.DataFrame({
    'product_id': df_pivot.index,
    'cluster_dtw': labels
})
df_clusters.to_csv("dtw_clusterstotal.csv", index=False)
print("✅ Archivo dtw_clusterstotal.csv generado correctamente.")


c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\waldo\miniconda3\envs\tslearn_env\lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all

✅ Archivo dtw_clusters40.csv generado correctamente.


In [29]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error


df['periodo'] = pd.to_datetime(df['periodo'])
df_clusters = pd.read_csv("dtw_clusterstotal.csv")  # columnas: product_id, cluster_dtw

# 2. Agrupar por producto y mes
df_prod = df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# 3. Agregar cluster
df_prod = df_prod.merge(df_clusters, on='product_id', how='left')

# 4. Promedio histórico de febreros (2017–2019)
feb_hist = df_prod[
    (df_prod['periodo'].dt.month == 2) & (df_prod['periodo'].dt.year < 2020)
].groupby('product_id')['tn'].mean().reset_index()
feb_hist.rename(columns={'tn': 'febrero_hist'}, inplace=True)

# 5. Media móvil 12 meses
df_prod = df_prod.sort_values(['product_id', 'periodo'])
df_prod['media_movil_12m'] = (
    df_prod.groupby('product_id')['tn']
    .transform(lambda x: x.rolling(window=12, min_periods=1).mean())
)

# 6. Crear lags
def crear_lags(df, lags=[1, 2, 3]):
    df_lags = df.copy()
    for lag in lags:
        df_lags[f'lag_{lag}'] = df_lags.groupby('product_id')['tn'].shift(lag)
    return df_lags

df_lags = crear_lags(df_prod)
df_lags['mes'] = df_lags['periodo'].dt.month

# 7. Features derivados
df_lags['var_lag1_lag2'] = df_lags['lag_1'] / df_lags['lag_2']
df_lags['delta_lag1_lag2'] = df_lags['lag_1'] - df_lags['lag_2']
df_lags['delta_rel_lag1_lag2'] = df_lags['delta_lag1_lag2'] / df_lags['lag_2']
df_lags['delta_ma12_lag1'] = df_lags['lag_1'] - df_lags['media_movil_12m']
df_lags.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

# 8. Preparar entrenamiento
df_train = df_lags.dropna()

features = [
    'lag_1', 'lag_2', 'lag_3', 'mes',
    'media_movil_12m', 'var_lag1_lag2',
    'delta_lag1_lag2', 'delta_rel_lag1_lag2', 'delta_ma12_lag1',
    'cluster_dtw'
]
target = 'tn'

train_data = df_train[df_train['periodo'] < '2019-12-01']
val_data = df_train[df_train['periodo'] == '2019-12-01']

X_train = train_data[features].copy()
y_train = train_data[target]
X_val = val_data[features].copy()
y_val = val_data[target]

# Conversión a float32 explícita
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# 9. Entrenar modelo
mejores_params = {
    'n_estimators': 761,
    'learning_rate': 0.03353760246239983,
    'max_depth': 14,
    'num_leaves': 224,
    'min_data_in_leaf': 44,
    'min_child_weight': 2.805764625749064,
    'subsample': 0.652624440966687,
    'subsample_freq': 6,
    'colsample_bytree': 0.943097954731396,
    'colsample_bynode': 0.8153611517204382,
    'reg_alpha': 3.935599636121298,
    'reg_lambda': 1.3996301457313742,
    'max_bin': 462,
    'min_split_gain': 0.8091677089451634,
    'cat_smooth': 65.43585455712281,
    'random_state': 42,
    'boosting_type':  'dart',
    'verbosity': -1,
    
    'linear_tree': True
    
}
modelo = lgb.LGBMRegressor(**mejores_params)
modelo.fit(X_train, y_train)

# 10. Validación
y_pred_val = modelo.predict(X_val)
print("📊 MAE en diciembre 2019:", mean_absolute_error(y_val, y_pred_val))

# 11. Preparar predicción febrero 2020
feb_df = pd.DataFrame()
feb_df['product_id'] = df_prod['product_id'].unique()
feb_df['periodo'] = pd.to_datetime('2020-02-01')
feb_df['mes'] = 2

lag1_df = df_prod[df_prod['periodo'] == '2019-12-01'][['product_id', 'media_movil_12m']].rename(columns={'media_movil_12m': 'lag_1'})
lag2 = df_prod[df_prod['periodo'] == '2019-12-01'][['product_id', 'tn']].rename(columns={'tn': 'lag_2'})
lag3 = df_prod[df_prod['periodo'] == '2019-11-01'][['product_id', 'tn']].rename(columns={'tn': 'lag_3'})
media_mov = df_prod[df_prod['periodo'] == '2019-12-01'][['product_id', 'media_movil_12m']]
clusters = df_clusters.copy()

feb_df = feb_df.merge(lag1_df, on='product_id')
feb_df = feb_df.merge(lag2, on='product_id')
feb_df = feb_df.merge(lag3, on='product_id')
feb_df = feb_df.merge(media_mov, on='product_id')
feb_df = feb_df.merge(clusters, on='product_id')

feb_df['var_lag1_lag2'] = feb_df['lag_1'] / feb_df['lag_2']
feb_df['delta_lag1_lag2'] = feb_df['lag_1'] - feb_df['lag_2']
feb_df['delta_rel_lag1_lag2'] = feb_df['delta_lag1_lag2'] / feb_df['lag_2']
feb_df['delta_ma12_lag1'] = feb_df['lag_1'] - feb_df['media_movil_12m']
feb_df = feb_df.merge(feb_hist, on='product_id', how='left')
feb_df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

# Conversión a float32 para predicción
X_feb = feb_df[features].astype('float32')

# 12. Predicción
feb_df['tn_predicho'] = modelo.predict(X_feb)
feb_df['tn_predicho_ajustada'] = 0.65 * feb_df['tn_predicho'] + 0.35 * feb_df['febrero_hist']

# 13. Exportar
#resultado = feb_df[['product_id', 'tn_predicho_ajustada']].rename(columns={'tn_predicho_ajustada': 'tn_predicho'})
resultado = feb_df[['product_id', 'tn_predicho_ajustada']].copy()
resultado['tn_predicho'] = resultado['tn_predicho_ajustada'] * 1.08
resultado = resultado[['product_id', 'tn_predicho']]

resultado.to_csv("prediccion_feb2020_datasettotal14.csv", index=False)
print("Predicción febrero 2020 guardada")


📊 MAE en diciembre 2019: 7.633826000213443
Predicción febrero 2020 guardada


In [28]:
import pandas as pd

# 1. Cargar predicción de febrero 2020
df_pred = pd.read_csv("prediccion_feb2020_datasettotal13.csv")  # archivo generado por el modelo
df_pred['periodo'] = pd.to_datetime("2020-02-01")
df_pred.rename(columns={'tn_predicho': 'tn'}, inplace=True)

# 2. Cargar dataset original
df_real = pd.read_csv("dataset_base_features.csv")
df_real["periodo"] = pd.to_datetime(df_real["periodo"])

# 3. Agrupar por período y sumar toneladas reales
tn_reales = df_real.groupby(["periodo"])["tn"].sum().reset_index()

# 4. Sumar toneladas predichas para febrero 2020
tn_feb2020 = df_pred.groupby("periodo")["tn"].sum().reset_index()

# 5. Unir y ordenar
comparacion = pd.concat([tn_reales, tn_feb2020], ignore_index=True)
comparacion = comparacion.sort_values("periodo").reset_index(drop=True)

# 6. Mostrar resultado
print("Total de toneladas por período (incluye predicción febrero 2020):")
print(comparacion.tail(36))  # los últimos 12 períodos


Total de toneladas por período (incluye predicción febrero 2020):
      periodo           tn
1  2017-02-01  27304.19010
2  2017-03-01  36656.74544
3  2017-04-01  31076.55205
4  2017-05-01  35808.35003
5  2017-06-01  35277.88414
6  2017-07-01  29462.94826
7  2017-08-01  32342.33769
8  2017-09-01  39281.85454
9  2017-10-01  40826.54749
10 2017-11-01  35676.87853
11 2017-12-01  29759.67068
12 2018-01-01  24856.68239
13 2018-02-01  28185.20277
14 2018-03-01  40064.32328
15 2018-04-01  30734.47379
16 2018-05-01  34238.52655
17 2018-06-01  30054.48264
18 2018-07-01  28305.34752
19 2018-08-01  31832.45606
20 2018-09-01  28706.06504
21 2018-10-01  32096.22497
22 2018-11-01  30170.71443
23 2018-12-01  24028.32642
24 2019-01-01  26406.14976
25 2019-02-01  27199.65581
26 2019-03-01  31955.92744
27 2019-04-01  30922.63637
28 2019-05-01  30984.42521
29 2019-06-01  30775.73369
30 2019-07-01  31546.44658
31 2019-08-01  23967.31272
32 2019-09-01  32704.44205
33 2019-10-01  34905.46581
34 2019-11-01  3